In [1]:
import numpy as np
import tensorflow as tf

In [2]:
data = np.load('./mcf7.data.npy')
# 166 bits fingerprint, 1 concentration float, 1 TGI float
unique_fp = np.load('./mcf7.unique.fp.npy')
# there is 6252 unique fingerprints and multiple experiments with each

In [3]:
np.random.shuffle(data)
test_data, train_data = np.vsplit(data, [100])

In [4]:
pretrain_batch_size = 512
batch_size = 64
latent_space = 4
input_space = 166
learning_rate = 0.01

In [5]:
def sample_prior(size=(batch_size, latent_space)):
    return np.random.normal(size=size)

def batch_gen(data, batch_size=batch_size):
    max_index = data.shape[0]/batch_size

    while True:
        np.random.shuffle(data)
        for i in xrange(max_index):
            yield np.hsplit(data[batch_size*i:batch_size*(i+1)], [-2, -1])

def same_gen(unq_fp, n_examples=batch_size, n_different=1, mu=-5.82, std=1.68):
    '''
    Generator of same fingerprints with different concentraition
    '''
    
    if n_examples % n_different: 
        raise ValueError('n_examples(%s) must be divisible by n_different(%s)' % (n_examples, n_different))
    max_index = unq_fp.shape[0] / n_different
    targets = np.zeros((n_examples, n_examples))
    block_size = n_examples/n_different
    for i in xrange(n_different):
        '''blocks of ones for every block of equal fp's'''
        targets[i*block_size:(i+1)*block_size, i*block_size:(i+1)*block_size] = 1.
    targets = targets > 0
    
    while 1:
        np.random.shuffle(unq_fp)
        for i in xrange(max_index):
            batch_conc = np.random.normal(mu, std, size=(n_examples, 1))
            batch_fp = np.repeat(unq_fp[i*n_different:(i+1)*n_different], [block_size]*n_different, axis=0)
            yield batch_fp, batch_conc, targets

            

In [6]:
fingerprint_tensor = tf.placeholder(tf.float32, [None, input_space])
prior_tensor = tf.placeholder(tf.float32, [None, latent_space])
conc_tensor = tf.placeholder(tf.float32, [None, 1])
tgi_tensor = tf.placeholder(tf.float32, [None, 1])
targets_tensor = tf.placeholder(tf.bool, [None, None])

visible_tensor = tf.concat(1, [fingerprint_tensor, conc_tensor])
hidden_tensor = tf.concat(1, [prior_tensor, tgi_tensor])

In [7]:
# Encoder net is 166+1->128->64->3+1
enc_w = {}
eff_w = {}
enc_w['w1'] = tf.Variable(tf.random_normal([input_space+1, 128]))
enc_w['w2'] = tf.Variable(tf.random_normal([128, 64]))
enc_w['w3'] = tf.Variable(tf.random_normal([64, latent_space]))
enc_w['b1'] = tf.Variable(tf.random_normal([128]))
enc_w['b2'] = tf.Variable(tf.random_normal([64]))
enc_w['b3'] = tf.Variable(tf.random_normal([latent_space]))
eff_w['w'] = tf.Variable(tf.random_normal([64, 1]))
eff_w['b'] = tf.Variable(tf.random_normal([1]))

In [8]:
# Decoder net 3+1->64->128->166+1
dec_w = {}
dec_w['w1'] = tf.Variable(tf.random_normal([latent_space+1, 64]))
dec_w['w2'] = tf.Variable(tf.random_normal([64, 128]))
dec_w['w3'] = tf.Variable(tf.random_normal([128, input_space]))
dec_w['b1'] = tf.Variable(tf.random_normal([64]))
dec_w['b2'] = tf.Variable(tf.random_normal([128]))
dec_w['b3'] = tf.Variable(tf.random_normal([input_space]))
dec_w['conc_w'] = tf.Variable(tf.random_normal([128, 1]))
dec_w['conc_b'] = tf.Variable(tf.random_normal([1]))

In [9]:
# Discriminator net 3->64->3->1
disc_w = {}
disc_w['w1'] = tf.Variable(tf.random_normal([latent_space, 2*latent_space]))
#disc_w['w2'] = tf.Variable(tf.random_normal([64, latent_space]))
disc_w['w3'] = tf.Variable(tf.random_normal([2*latent_space, 1]))
disc_w['b1'] = tf.Variable(tf.random_normal([2*latent_space]))
#disc_w['b2'] = tf.Variable(tf.random_normal([latent_space]))
disc_w['b3'] = tf.Variable(tf.random_normal([1]))

In [10]:
enc_l1 = tf.nn.tanh(tf.add(tf.matmul(visible_tensor, enc_w['w1']), enc_w['b1']))
enc_l2 = tf.nn.tanh(tf.add(tf.matmul(enc_l1, enc_w['w2']), enc_w['b2']))
encoded_fp = tf.add(tf.matmul(enc_l2, enc_w['w3']), enc_w['b3'])
encoded_tgi = tf.add(tf.matmul(enc_l2, eff_w['w']), eff_w['b'])

encoded = tf.concat(1, [encoded_fp, encoded_tgi])

dec_l1 = tf.nn.tanh(tf.add(tf.matmul(encoded, dec_w['w1']), dec_w['b1']))
dec_l2 = tf.nn.tanh(tf.add(tf.matmul(dec_l1, dec_w['w2']), dec_w['b2']))
decoded_fp = tf.add(tf.matmul(dec_l2, dec_w['w3']), dec_w['b3'])
decoded_conc = tf.add(tf.matmul(dec_l2, dec_w['conc_w']), dec_w['conc_b'])

disc_enc_l1 = tf.nn.sigmoid(tf.add(tf.matmul(encoded_fp, disc_w['w1']), disc_w['b1']))
#disc_enc_l2 = tf.nn.sigmoid(tf.add(tf.matmul(disc_enc_l1, disc_w['w2']), disc_w['b2']))
disc_enc = tf.add(tf.matmul(disc_enc_l1, disc_w['w3']), disc_w['b3'])

disc_prior_l1 = tf.nn.sigmoid(tf.add(tf.matmul(prior_tensor, disc_w['w1']), disc_w['b1']))
#disc_prior_l2 = tf.nn.sigmoid(tf.add(tf.matmul(disc_prior_l1, disc_w['w2']), disc_w['b2']))
disc_prior = tf.add(tf.matmul(disc_prior_l1, disc_w['w3']), disc_w['b3'])


In [11]:
gen_l1 = tf.nn.tanh(tf.add(tf.matmul(hidden_tensor, dec_w['w1']), dec_w['b1']))
gen_l2 = tf.nn.tanh(tf.add(tf.matmul(gen_l1, dec_w['w2']), dec_w['b2']))
gen_fp = tf.nn.sigmoid(tf.add(tf.matmul(gen_l2, dec_w['w3']), dec_w['b3']))
gen_conc = tf.add(tf.matmul(gen_l2, dec_w['conc_w']), dec_w['conc_b'])

In [12]:
disc_loss = tf.reduce_mean(tf.nn.relu(disc_prior) - disc_prior + tf.log(1.0 + tf.exp(-tf.abs(disc_prior)))) + \
            tf.reduce_mean(tf.nn.relu(disc_enc) + tf.log(1.0 + tf.exp(-tf.abs(disc_enc))))

fp_norms = tf.sqrt(tf.reduce_sum(tf.square(encoded_fp), keep_dims=True, reduction_indices=[1]))
normalized_fp = tf.div(encoded_fp, fp_norms)
cosines_fp = tf.matmul(normalized_fp, tf.transpose(normalized_fp))
manifold_cost = tf.reduce_mean(1 - tf.boolean_mask(cosines_fp, targets_tensor))
    
enc_fp_loss = tf.reduce_mean(tf.nn.relu(disc_enc) - disc_enc + tf.log(1.0 + tf.exp(-tf.abs(disc_enc))))
enc_tgi_loss = tf.reduce_mean(tf.square(tf.sub(tgi_tensor, encoded_tgi)))
enc_loss = enc_fp_loss

dec_fp_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(decoded_fp, fingerprint_tensor))
dec_conc_loss = tf.reduce_mean(tf.square(tf.sub(conc_tensor, decoded_conc)))
dec_loss = dec_fp_loss + dec_conc_loss

In [13]:
# train_discriminator = tf.train.AdagradOptimizer(learning_rate).minimize(disc_loss, var_list=disc_w.values())
# train_encoder = tf.train.AdagradOptimizer(learning_rate).minimize(enc_loss, var_list=enc_w.values())
# train_manifold = tf.train.AdagradOptimizer(learning_rate).minimize(manifold_cost, var_list=enc_w.values())
# train_reg = tf.train.AdagradOptimizer(learning_rate).minimize(enc_tgi_loss, var_list=enc_w.values()+eff_w.values())
# train_autoencoder = tf.train.AdagradOptimizer(learning_rate).minimize(dec_loss, var_list=enc_w.values()+eff_w.values()+dec_w.values())

train_discriminator = tf.train.AdamOptimizer(learning_rate).minimize(disc_loss, var_list=disc_w.values())
train_encoder = tf.train.AdamOptimizer(learning_rate).minimize(enc_loss, var_list=enc_w.values())
train_manifold = tf.train.AdamOptimizer(learning_rate).minimize(manifold_cost, var_list=enc_w.values())
train_reg = tf.train.AdamOptimizer(learning_rate).minimize(enc_tgi_loss, var_list=enc_w.values()+eff_w.values())
train_autoencoder = tf.train.AdamOptimizer(learning_rate).minimize(dec_loss, var_list=enc_w.values()+eff_w.values()+dec_w.values())

/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [14]:
init = tf.initialize_all_variables()

In [15]:
sess = tf.Session()
saver = tf.train.Saver()

In [16]:
batches = batch_gen(train_data, pretrain_batch_size)
sames = same_gen(unique_fp, n_different=32)

In [18]:
# pretrain generator w/o regressions and decoding
flag = True
while flag:
    # need to do a few initialization tries, because from some points
    # Generator doesn't converge.
    
    sess.run(init)
    for e in xrange(15):
        print("epoch #%d" % e)
        discriminator_loss = 0.0
        encoder_fp_loss = 0.0
        mani_loss = 0.0
        for u in xrange(1000):
            batch_fp, batch_conc, _ = batches.next()
            batch_prior = sample_prior()
            _, loss = sess.run([train_discriminator, disc_loss], feed_dict={fingerprint_tensor: batch_fp,
                                                                            conc_tensor: batch_conc,
                                                                            prior_tensor: batch_prior})
            discriminator_loss += loss

            fp_loss = 2.
            count = 0
            while fp_loss > 1. and count < 20:
                batch_fp, batch_conc, _ = batches.next()
                _, fp_loss = sess.run([train_encoder, enc_fp_loss], feed_dict={fingerprint_tensor: batch_fp,
                                                                               conc_tensor: batch_conc,})
                count += 1
            else:
                encoder_fp_loss += fp_loss

            same_fp, same_conc, targets = sames.next()
            _, m_loss = sess.run([train_manifold, manifold_cost], feed_dict={fingerprint_tensor: batch_fp,
                                                                             conc_tensor: batch_conc,
                                                                             targets_tensor: targets})
            mani_loss += m_loss

        discriminator_loss /= 1000. * 2.
        encoder_fp_loss /= 1000.
        mani_loss /= 1000.

        print("disc: %f, enc_p: %f, mani: %f" % (discriminator_loss, encoder_fp_loss, mani_loss))
        if (e >= 5) and (encoder_fp_loss < 0.7):
            flag = False
            break

epoch #0
disc: 0.190337, enc_p: 2.408769, mani: 0.002079
epoch #1
disc: 0.016269, enc_p: 4.458658, mani: 0.000000
epoch #2
disc: 0.003061, enc_p: 5.737958, mani: -0.000000
epoch #3
disc: 0.001393, enc_p: 6.522139, mani: 0.000000
epoch #4
disc: 0.000729, enc_p: 7.165398, mani: 0.000000
epoch #5
disc: 0.000407, enc_p: 7.746550, mani: 0.000000
epoch #6
disc: 0.000235, enc_p: 8.294210, mani: -0.000000
epoch #7
disc: 0.000138, enc_p: 8.824687, mani: 0.000000
epoch #8
disc: 0.000082, enc_p: 9.345799, mani: -0.000000
epoch #9
disc: 0.000049, enc_p: 9.862336, mani: 0.000000
epoch #10
disc: 0.000029, enc_p: 10.380089, mani: 0.000000
epoch #11
disc: 0.000017, enc_p: 10.903279, mani: 0.000000
epoch #12
disc: 0.000010, enc_p: 11.438785, mani: 0.000000
epoch #13
disc: 0.000006, enc_p: 11.993212, mani: -0.000000
epoch #14
disc: 0.000005, enc_p: 12.083346, mani: -0.000000
epoch #0
disc: 0.293447, enc_p: 1.983980, mani: 0.001609
epoch #1
disc: 0.010665, enc_p: 4.690299, mani: 0.000000
epoch #2
disc: 0

In [19]:
batches = batch_gen(train_data, batch_size)
sames = same_gen(unique_fp, n_different=32)
for e in xrange(50):
    print("epoch #%d" % e)
    
    for u in xrange(10000):
        batch_fp, batch_conc, batch_tgi = batches.next()
        batch_prior = sample_prior()
        sess.run(train_discriminator, feed_dict={fingerprint_tensor: batch_fp,
                                                 conc_tensor: batch_conc,
                                                 tgi_tensor: batch_tgi,
                                                 prior_tensor: batch_prior})

        batch_fp, batch_conc, batch_tgi = batches.next()

        sess.run(train_encoder, feed_dict={fingerprint_tensor: batch_fp,
                                           conc_tensor: batch_conc})

#         same_fp, same_conc, targets = sames.next()
#         sess.run(train_manifold, feed_dict={fingerprint_tensor: batch_fp,
#                                             conc_tensor: batch_conc,
#                                             targets_tensor: targets})
    
        batch_fp, batch_conc, batch_tgi = batches.next()
        sess.run(train_reg, feed_dict={fingerprint_tensor: batch_fp,
                                       conc_tensor: batch_conc,
                                       tgi_tensor: batch_tgi})

        batch_fp, batch_conc, batch_tgi = batches.next()
        sess.run(train_autoencoder, feed_dict={fingerprint_tensor: batch_fp,
                                               conc_tensor: batch_conc,
                                               tgi_tensor: batch_tgi})
        
    else:
        batch_prior = sample_prior((100, latent_space))
        losses = sess.run([disc_loss, enc_fp_loss, enc_tgi_loss, dec_fp_loss, dec_conc_loss],
                          feed_dict={fingerprint_tensor: train_data[:, :-2],
                                     conc_tensor: train_data[:, -2:-1],
                                     tgi_tensor: train_data[:, -1:],
                                     prior_tensor: batch_prior
                                    })
        
        same_fp, same_conc, targets = sames.next()
        m_loss = sess.run(manifold_cost, feed_dict={fingerprint_tensor: batch_fp,
                                                    conc_tensor: batch_conc,
                                                    targets_tensor: targets})
        
        discriminator_loss, encoder_fp_loss, encoder_tgi_loss, autoencoder_fp_loss, autoencoder_conc_loss = losses
        print("disc: %f, enc_fp : %f, mani_fp: %f, enc_tgi: %f, dec_fp : %f, dec_conc : %f" % (discriminator_loss/2.,
                                                                                               encoder_fp_loss,
                                                                                               m_loss,
                                                                                               encoder_tgi_loss,
                                                                                               autoencoder_fp_loss,
                                                                                               autoencoder_conc_loss))

epoch #0
disc: 0.693160, enc_fp : 0.688109, mani_fp: 0.000000, enc_tgi: 3139.656006, dec_fp : 0.437040, dec_conc : 2.852242
epoch #1
disc: 0.693346, enc_fp : 0.673428, mani_fp: 0.000000, enc_tgi: 3141.742188, dec_fp : 0.434595, dec_conc : 2.935303
epoch #2
disc: 0.693147, enc_fp : 0.693253, mani_fp: 0.000000, enc_tgi: 3138.131348, dec_fp : 0.431563, dec_conc : 2.970067
epoch #3
disc: 0.693147, enc_fp : 0.693231, mani_fp: 0.000000, enc_tgi: 3137.025391, dec_fp : 0.434331, dec_conc : 2.848898
epoch #4
disc: 0.693147, enc_fp : 0.693091, mani_fp: 0.000000, enc_tgi: 3136.597168, dec_fp : 0.437752, dec_conc : 2.942320
epoch #5
disc: 0.693147, enc_fp : 0.693067, mani_fp: 0.000000, enc_tgi: 3136.181152, dec_fp : 0.439213, dec_conc : 2.848863
epoch #6
disc: 0.693147, enc_fp : 0.692878, mani_fp: 0.000000, enc_tgi: 3136.729492, dec_fp : 0.434631, dec_conc : 2.874536
epoch #7
disc: 0.693147, enc_fp : 0.692960, mani_fp: 0.000000, enc_tgi: 3140.440674, dec_fp : 0.435014, dec_conc : 2.847920
epoch #8

KeyboardInterrupt: 

In [21]:
saver.save(sess, './4dN.2L.adam.aae.manifold.5e.model.ckpt')

'./4dN.2L.adam.aae.manifold.5e.model.ckpt'

In [18]:
saver.restore(sess, './4dN.2L.128.64.aae.manifold.5e.model.ckpt')

In [22]:
prior_batch = sample_prior(size=(1000, latent_space))

In [23]:
batch_tgi = np.ones((1000, 1)) * 5. + np.random.normal(0., 5., size=(1000, 1))

In [24]:
generated_fp, generated_conc = sess.run([gen_fp, gen_conc], 
                                        feed_dict={prior_tensor: prior_batch, tgi_tensor: batch_tgi})

In [25]:
res = np.hstack([generated_fp, generated_conc, batch_tgi])

In [26]:
np.save('./generated.probs.5e.tgi5', res)

In [26]:
generated_fp.shape

(1000, 166)

In [54]:
sampled = (np.hstack([(generated_fp > 0.5).astype(np.int), generated_conc, batch_tgi]))

In [55]:
np.save('./generated.sample.50e', sampled)

In [56]:
np.sum(generated_conc < -5)

220

In [57]:
np.sum(batch_tgi < 0)

982

In [63]:
np.sum((generated_conc < -5) & (batch_tgi < 0))

218